In [1]:
import warnings
import pandas as pd
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')


In [2]:
df1=pd.read_csv('area_passenger_index.csv',header=None
                ,names=['ID','time','people_index'])
df2=pd.read_csv('area_passenger_info.csv',header=None
                ,names=['ID','area_name','area_type','Center_x','Center_y','Grid_x','Grid_y','area'])
df3=pd.read_csv('migration_index.csv',header=None
            ,names=['date','departure_province','departure_city','arrival_province','arrival_city','index'])
df4=pd.read_csv('datafountain_competition_od.txt',sep='\t',header=None
                ,names=['hour','start_grid_x','start_grid_y','end_grid_x','end_grid_y','Index'])

df5 = pd.read_csv('area_relation2.csv')  #
print("df columns:",df5.columns)
sub=pd.read_csv('test_submit_example.csv',header=None
                ,names=['ID','time','people_index'])
df5.head(10)

df columns: Index(['Unnamed: 0', 'area_id', 'hour', 'r_in', 'r_out', 'r_self', 'r_all'], dtype='object')


,Unnamed: 0,area_id,hour,r_in,r_out,r_self,r_all
0,0,0,0.0,353.5,312.9,3637.9,4304.3
1,1,1,0.0,0.4,0.4,0.0,0.8
2,2,2,0.0,0.8,1.2,0.0,2.0
3,3,3,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,0.0,0.0,0.0,0.0
5,5,5,0.0,0.0,0.0,0.0,0.0
6,6,6,0.0,0.0,0.0,0.0,0.0
7,7,7,0.0,0.1,0.0,0.0,0.1
8,8,8,0.0,0.1,0.1,0.0,0.2
9,9,9,0.0,0.5,0.5,0.0,1.0


In [3]:
df1['time']=pd.to_datetime(df1['time'],format='%Y%m%d%H')
df1['day']=df1['time'].dt.day.apply(lambda x:x-17 if x>=17 else x+14)
df1['weekday']=df1['time'].dt.weekday
df1['hour']=df1['time'].dt.hour

In [4]:
dict_wd=dict(df1.groupby(['weekday']).mean()['people_index'])
dict_h=dict(df1.groupby(['hour']).mean()['people_index'])
dict_id=dict(df1.groupby(['ID']).mean()['people_index'])

In [5]:
df1['weekday_encoded']=df1['weekday'].map(dict_wd)
df1['hour_encoded']=df1['hour'].map(dict_h)
df1['ID_encoded']=df1['ID'].map(dict_id)

# add realtion

In [6]:
df5 = df5[['area_id','hour','r_in','r_out','r_self','r_all']]
df5_values = df5.values
relation_encoder = {}
for m in df5_values:
    relation_encoder[(m[0],m[1])] = [m[2],m[3],m[4],m[5]]
    
df1_id_h = df1[['ID','hour']].values
mm = []
for x in df1_id_h:
    mm.append(relation_encoder[(x[0],x[1])])
mm = np.array(mm).T
df1['r_in'] = pd.DataFrame(mm[0])
df1['r_out'] = pd.DataFrame(mm[1])
df1['r_self'] = pd.DataFrame(mm[2])
df1['r_all'] = pd.DataFrame(mm[3])

In [7]:
df1.head(10)

,ID,time,people_index,day,weekday,hour,weekday_encoded,hour_encoded,ID_encoded,r_in,r_out,r_self,r_all
0,1,2020-01-17 00:00:00,1.8,0,4,0,28.266556,12.499074,4.835556,0.4,0.4,0.0,0.8
1,1,2020-01-17 01:00:00,1.5,0,4,1,28.266556,8.617108,4.835556,0.8,0.2,0.0,1.0
2,1,2020-01-17 02:00:00,1.3,0,4,2,28.266556,7.245714,4.835556,0.1,0.2,0.0,0.3
3,1,2020-01-17 03:00:00,1.3,0,4,3,28.266556,6.373882,4.835556,0.3,0.7,0.0,1.0
4,1,2020-01-17 04:00:00,1.7,0,4,4,28.266556,6.205978,4.835556,0.8,0.7,0.0,1.5
5,1,2020-01-17 05:00:00,1.2,0,4,5,28.266556,7.780906,4.835556,1.0,1.1,0.0,2.1
6,1,2020-01-17 06:00:00,2.9,0,4,6,28.266556,13.849278,4.835556,2.3,4.2,0.0,6.5
7,1,2020-01-17 07:00:00,4.7,0,4,7,28.266556,22.833651,4.835556,4.5,7.6,0.0,12.1
8,1,2020-01-17 08:00:00,6.1,0,4,8,28.266556,31.165440,4.835556,3.1,7.7,0.0,10.8
9,1,2020-01-17 09:00:00,5.2,0,4,9,28.266556,35.761210,4.835556,3.5,3.8,0.0,7.3


# add

In [8]:
# #  add by zhangyang ######
dict_wd1=dict(df1.groupby(['weekday']).median()['people_index'])
dict_h1=dict(df1.groupby(['hour']).median()['people_index'])
dict_id1=dict(df1.groupby(['ID']).median()['people_index'])
df1['weekday_encoded1']=df1['weekday'].map(dict_wd1)
df1['hour_encoded1']=df1['hour'].map(dict_h1)
df1['ID_encoded1']=df1['ID'].map(dict_id1)

In [9]:
pd.set_option('display.width',None)

In [10]:
train_data=df1.copy()

In [11]:
train_data=pd.merge(df1,df2,on=['ID'])

In [12]:
dict_at=dict(train_data.groupby(['area_type']).mean()['people_index'])

In [13]:
train_data['area_type_encoded']=train_data['area_type'].map(dict_at)

# add

In [14]:
# add by zhangyang #########
dict_at1=dict(train_data.groupby(['area_type']).median()['people_index'])
train_data['area_type_encoded1']=train_data['area_type'].map(dict_at1)

In [15]:
train_data.drop(['area_name','area_type'],axis=1,inplace=True)
train_data.head()

,ID,time,people_index,day,weekday,hour,weekday_encoded,hour_encoded,ID_encoded,r_in,...,weekday_encoded1,hour_encoded1,ID_encoded1,Center_x,Center_y,Grid_x,Grid_y,area,area_type_encoded,area_type_encoded1
0,1,2020-01-17 00:00:00,1.8,0,4,0,28.266556,12.499074,4.835556,0.4,...,6.5,3.8,4.1,116.256586,39.986913,116.256713,39.987525,395925.609375,10.071021,3.4
1,1,2020-01-17 01:00:00,1.5,0,4,1,28.266556,8.617108,4.835556,0.8,...,6.5,2.6,4.1,116.256586,39.986913,116.256713,39.987525,395925.609375,10.071021,3.4
2,1,2020-01-17 02:00:00,1.3,0,4,2,28.266556,7.245714,4.835556,0.1,...,6.5,2.1,4.1,116.256586,39.986913,116.256713,39.987525,395925.609375,10.071021,3.4
3,1,2020-01-17 03:00:00,1.3,0,4,3,28.266556,6.373882,4.835556,0.3,...,6.5,1.8,4.1,116.256586,39.986913,116.256713,39.987525,395925.609375,10.071021,3.4
4,1,2020-01-17 04:00:00,1.7,0,4,4,28.266556,6.205978,4.835556,0.8,...,6.5,1.7,4.1,116.256586,39.986913,116.256713,39.987525,395925.609375,10.071021,3.4


In [16]:
df3['date']=pd.to_datetime(df3['date'],format='%Y%m%d')
df3['day_encode']=df3['date'].dt.day.apply(lambda x:x-17 if x>=17 else x+14)
df3['city']=df3['arrival_city'].apply(lambda x:1 if x=='北京市' else 0)

inbj=df3[df3['city']==1]
outbj=df3[df3['city']==0]
index_mean_in=inbj.groupby(['date']).mean()['index']
index_std_in=inbj.groupby(['date']).std()['index']
index_mean_in_copy=index_mean_in.copy()
index_mean_in_copy.index=np.arange(30)
dict4=dict(index_mean_in_copy)
index_std_in_copy=index_std_in.copy()
index_std_in_copy.index=np.arange(30)
dict5=dict(index_std_in_copy)
index_mean_out=outbj.groupby(['date']).mean()['index']
index_std_out=outbj.groupby(['date']).std()['index']
index_mean_out_copy=index_mean_out.copy()
index_mean_out_copy.index=np.arange(30)
dict6=dict(index_mean_out_copy)
index_std_out_copy=index_std_out.copy()
index_std_out_copy.index=np.arange(30)
dict7=dict(index_std_out_copy)
train_data['index_mean_in']=train_data['day'].map(dict4)
train_data['index_std_in']=train_data['day'].map(dict5)
train_data['index_mean_out']=train_data['day'].map(dict6)
train_data['index_std_out']=train_data['day'].map(dict7)

In [17]:
ar_model1=AR(index_mean_in).fit()
pred_index_mean_in=ar_model1.predict('20200216','20200224',dynamic=True)
ar_model3=AR(index_mean_out).fit()
pred_index_mean_out=ar_model3.predict('20200216','20200224',dynamic=True)
ar_model2=AR(index_std_in).fit()
pred_index_std_in=ar_model2.predict('20200216','20200224',dynamic=True)
ar_model4=AR(index_std_out).fit()
pred_index_std_out=ar_model4.predict('20200216','20200224',dynamic=True)
dict_hi=dict(df4.groupby(['hour']).mean()['Index'])
train_data['hi_encoded']=train_data['hour'].map(dict_hi)

In [18]:
test_data=sub.copy()
test_data.drop(['people_index'],axis=1,inplace=True)
test_data['time']=pd.to_datetime(test_data['time'],format='%Y%m%d%H')
test_data['weekday']=test_data['time'].dt.weekday
test_data['hour']=test_data['time'].dt.hour

# add relation to test data
test_data_id_h = test_data[['ID','hour']].values
mm = []
for x in test_data_id_h:
    mm.append(relation_encoder[(x[0],x[1])])
mm = np.array(mm).T
test_data['r_in'] = pd.DataFrame(mm[0])
test_data['r_out'] = pd.DataFrame(mm[1])
test_data['r_self'] = pd.DataFrame(mm[2])
test_data['r_all'] = pd.DataFrame(mm[3])

test_data['weekday_encoded']=test_data['weekday'].map(dict_wd)
test_data['weekday_encoded1']=test_data['weekday'].map(dict_wd1)
test_data['hour_encoded']=test_data['hour'].map(dict_h)
test_data['hour_encoded1']=test_data['hour'].map(dict_h1)
test_data['ID_encoded']=test_data['ID'].map(dict_id)
test_data['ID_encoded1']=test_data['ID'].map(dict_id1)

test_data['day']=test_data['time'].dt.day.apply(lambda x:x+14)
pred_index_mean_in.index=np.arange(30,39,1)
dict_pim=dict(pred_index_mean_in)
pred_index_std_in.index=np.arange(30,39,1)
dict_pis=dict(pred_index_std_in)

pred_index_mean_out.index=np.arange(30,39,1)
dict_pim1=dict(pred_index_mean_out)
pred_index_std_out.index=np.arange(30,39,1)
dict_pis1=dict(pred_index_std_out)

In [19]:
test_data['index_mean_in']=test_data['day'].map(dict_pim)
test_data['index_std_in']=test_data['day'].map(dict_pis)
test_data['index_mean_out']=test_data['day'].map(dict_pim1)
test_data['index_std_out']=test_data['day'].map(dict_pis1)
test_data=pd.merge(test_data,df2,on=['ID'])
test_data.drop(['Center_x','Center_y','Grid_x','Grid_y'],axis=1,inplace=True)
test_data['hi_encoded']=test_data['hour'].map(dict_hi)

test_data['area_type_encoded']=test_data['area_type'].map(dict_at)
test_data['area_type_encoded1']=test_data['area_type'].map(dict_at1)

test_data.drop(['ID','time','weekday','hour','day','area_name','area_type'],axis=1,inplace=True)
train_data=train_data[train_data['day']>=16]
train_data.index=np.arange(train_data.shape[0])
target=train_data['people_index']

In [20]:
train_data.drop(['people_index','day','ID','weekday','hour','Center_x','Center_y','Grid_x','Grid_y','time']
                ,axis=1,inplace=True)
print('train_data.head()\n',train_data.head())
test_new=pd.DataFrame([test_data['weekday_encoded']#周几平均
                     ,test_data['weekday_encoded1'] # 周中位数
                     ,test_data['hour_encoded']#小时平均
                     ,test_data['hour_encoded1']#x小时中位数
                     ,test_data['ID_encoded']#区域平均
                     ,test_data['ID_encoded1'] # 区域中位数
                     ,test_data['area']#区域面积
                     ,test_data['area_type_encoded']#区域类型平均
                     ,test_data['area_type_encoded1']
                     ,test_data['index_mean_in']#迁入平均
                     ,test_data['index_std_in']#迁入方差
                     ,test_data['index_mean_out']#迁出平均
                     ,test_data['index_std_out']#迁出方差
                     ,test_data['hi_encoded']
                     ,test_data['r_in']
                     ,test_data['r_out']
                      ,test_data['r_self']
                      ,test_data['r_all']]).T#网格间流动小时平均
print('test_new.head()\n',test_new.head())

train_data.head()
    weekday_encoded  hour_encoded  ID_encoded  r_in  r_out  r_self  r_all  \
0        27.552537     12.499074    4.835556   0.4    0.4     0.0    0.8   
1        27.552537      8.617108    4.835556   0.8    0.2     0.0    1.0   
2        27.552537      7.245714    4.835556   0.1    0.2     0.0    0.3   
3        27.552537      6.373882    4.835556   0.3    0.7     0.0    1.0   
4        27.552537      6.205978    4.835556   0.8    0.7     0.0    1.5   

   weekday_encoded1  hour_encoded1  ID_encoded1           area  \
0               6.2            3.8          4.1  395925.609375   
1               6.2            2.6          4.1  395925.609375   
2               6.2            2.1          4.1  395925.609375   
3               6.2            1.8          4.1  395925.609375   
4               6.2            1.7          4.1  395925.609375   

   area_type_encoded  area_type_encoded1  index_mean_in  index_std_in  \
0          10.071021                 3.4       0.01689

In [21]:
import copy
train_data_cp = copy.deepcopy(train_data)
test_new_cp = copy.deepcopy(test_new)

In [22]:
columns = list(train_data_cp.columns)
train_data = train_data_cp[columns]
test_new = test_new_cp[columns]
train_data = train_data.drop(['r_all'],axis=1) #['weekday_encoded','hour_encoded','ID_encoded']
# # train_data = train_data.drop(['r_in','r_out','r_self'],axis=1)
test_new = test_new.drop(['r_all'],axis=1)
train_data.head(10)

,weekday_encoded,hour_encoded,ID_encoded,r_in,r_out,r_self,weekday_encoded1,hour_encoded1,ID_encoded1,area,area_type_encoded,area_type_encoded1,index_mean_in,index_std_in,index_mean_out,index_std_out,hi_encoded
0,27.552537,12.499074,4.835556,0.4,0.4,0.0,6.2,3.8,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.111379
1,27.552537,8.617108,4.835556,0.8,0.2,0.0,6.2,2.6,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.110136
2,27.552537,7.245714,4.835556,0.1,0.2,0.0,6.2,2.1,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.109777
3,27.552537,6.373882,4.835556,0.3,0.7,0.0,6.2,1.8,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.107989
4,27.552537,6.205978,4.835556,0.8,0.7,0.0,6.2,1.7,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.106042
5,27.552537,7.780906,4.835556,1.0,1.1,0.0,6.2,2.1,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.105457
6,27.552537,13.849278,4.835556,2.3,4.2,0.0,6.2,3.8,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.105526
7,27.552537,22.833651,4.835556,4.5,7.6,0.0,6.2,6.5,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.106695
8,27.552537,31.165440,4.835556,3.1,7.7,0.0,6.2,9.1,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.107794
9,27.552537,35.761210,4.835556,3.5,3.8,0.0,6.2,10.5,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.107154


In [23]:
test_new.head(10)

,weekday_encoded,hour_encoded,ID_encoded,r_in,r_out,r_self,weekday_encoded1,hour_encoded1,ID_encoded1,area,area_type_encoded,area_type_encoded1,index_mean_in,index_std_in,index_mean_out,index_std_out,hi_encoded
0,27.552537,12.499074,4.835556,0.4,0.4,0.0,6.2,3.8,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.111379
1,27.552537,8.617108,4.835556,0.8,0.2,0.0,6.2,2.6,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.110136
2,27.552537,7.245714,4.835556,0.1,0.2,0.0,6.2,2.1,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.109777
3,27.552537,6.373882,4.835556,0.3,0.7,0.0,6.2,1.8,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.107989
4,27.552537,6.205978,4.835556,0.8,0.7,0.0,6.2,1.7,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.106042
5,27.552537,7.780906,4.835556,1.0,1.1,0.0,6.2,2.1,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.105457
6,27.552537,13.849278,4.835556,2.3,4.2,0.0,6.2,3.8,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.105526
7,27.552537,22.833651,4.835556,4.5,7.6,0.0,6.2,6.5,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.106695
8,27.552537,31.165440,4.835556,3.1,7.7,0.0,6.2,9.1,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.107794
9,27.552537,35.761210,4.835556,3.5,3.8,0.0,6.2,10.5,4.1,395925.609375,10.071021,3.4,0.004731,0.029124,0.002587,0.01497,0.107154


In [24]:
model_lgb=lgb.LGBMRegressor(num_leaves=20
                            ,max_depth=6
                            ,learning_rate=0.09
                            ,n_estimators=1100#最大生成树数量
                            ,n_jobs=-1)#所有core进行计算
x_train,x_test,y_train,y_test=train_test_split(train_data,target,test_size=0.2, shuffle=False)
model_lgb.fit(x_train,y_train)
y_hat=model_lgb.predict(x_test)
y_hat[y_hat<0]=0
print(1/(mean_squared_error(y_test,y_hat)**0.5+1))
keys=[]
sk=KFold(n_splits=10,shuffle=False)
for train,test in sk.split(train_data,target):
    x_train=train_data.iloc[train]
    y_train=target.iloc[train]
    x_test=train_data.iloc[test]
    y_test=target.iloc[test]
    model_lgb.fit(x_train,y_train)
    y_hat=model_lgb.predict(x_test)
    y_hat[y_hat<0]=0
    print(1/(mean_squared_error(y_test,y_hat)**0.5+1))

    keys.append(model_lgb.predict(test_new))
keys=np.array(keys)
# km=y_hat
# km[km<0]=0
# sub.iloc[:,2]=km

# sub.to_csv('result_relation2.csv',encoding='utf-8',header=None,index=None)


0.12387561865832812
0.05729824397005884
0.07453139654671918
0.08219572056803917
0.059274678120665024
0.14721337892223024
0.1318127947211612
0.11827457707405323
0.012986601373692364
0.15472308692190265
0.09254054444919191


In [32]:
# 取效果最好的三个
km=np.average(keys, axis=0, weights=[0, 0, 0, 0, 0.1472, 0.1318, 0, 0, 0.1547, 0])
km = model_lgb.predict(test_new)
km[km<0]=0
sub.iloc[:,2]=km

sub.to_csv('result_relation2.csv',encoding='utf-8',header=None,index=None)


In [25]:
x_train

,weekday_encoded,hour_encoded,ID_encoded,r_in,r_out,weekday_encoded1,hour_encoded1,ID_encoded1,area,area_type_encoded,area_type_encoded1,index_mean_in,index_std_in,index_mean_out,index_std_out,hi_encoded
0,27.552537,12.499074,4.835556,0.4,0.4,6.2,3.8,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.111379
1,27.552537,8.617108,4.835556,0.8,0.2,6.2,2.6,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.110136
3,27.552537,6.373882,4.835556,0.3,0.7,6.2,1.8,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.107989
4,27.552537,6.205978,4.835556,0.8,0.7,6.2,1.7,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.106042
5,27.552537,7.780906,4.835556,1.0,1.1,6.2,2.1,4.1,395925.609375,10.071021,3.4,0.016892,0.042052,0.003928,0.015576,0.105457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334987,26.135403,31.906556,0.001667,0.0,0.0,6.2,8.8,0.0,39963.562500,8.648456,2.4,0.004526,0.015719,0.002459,0.015534,0.108056
334988,26.135403,28.783407,0.001667,0.0,0.0,6.2,8.1,0.0,39963.562500,8.648456,2.4,0.004526,0.015719,0.002459,0.015534,0.108469
334989,26.135403,25.377081,0.001667,0.0,0.0,6.2,7.2,0.0,39963.562500,8.648456,2.4,0.004526,0.015719,0.002459,0.015534,0.108725
334990,26.135403,21.086499,0.001667,0.0,0.0,6.2,6.4,0.0,39963.562500,8.648456,2.4,0.004526,0.015719,0.002459,0.015534,0.109927


In [26]:
inbj.groupby(['date']).mean()['index']

date
2020-01-17    0.021259
2020-01-18    0.019660
2020-01-19    0.020686
2020-01-20    0.018919
2020-01-21    0.019067
2020-01-22    0.015993
2020-01-23    0.013219
2020-01-24    0.010207
2020-01-25    0.007061
2020-01-26    0.014135
2020-01-27    0.017364
2020-01-28    0.016684
2020-01-29    0.015595
2020-01-30    0.016910
2020-01-31    0.016037
2020-02-01    0.015355
2020-02-02    0.016892
2020-02-03    0.015540
2020-02-04    0.009758
2020-02-05    0.008186
2020-02-06    0.006952
2020-02-07    0.007326
2020-02-08    0.008160
2020-02-09    0.011226
2020-02-10    0.011894
2020-02-11    0.007470
2020-02-12    0.006291
2020-02-13    0.006087
2020-02-14    0.005023
2020-02-15    0.004526
Name: index, dtype: float64

In [27]:
inbj.groupby(['date']).median()['index']


date
2020-01-17    0.003450
2020-01-18    0.003430
2020-01-19    0.003370
2020-01-20    0.003270
2020-01-21    0.003920
2020-01-22    0.003660
2020-01-23    0.003690
2020-01-24    0.003080
2020-01-25    0.001750
2020-01-26    0.003080
2020-01-27    0.005090
2020-01-28    0.006400
2020-01-29    0.005765
2020-01-30    0.005990
2020-01-31    0.005605
2020-02-01    0.005410
2020-02-02    0.005900
2020-02-03    0.005480
2020-02-04    0.003500
2020-02-05    0.003050
2020-02-06    0.002675
2020-02-07    0.002490
2020-02-08    0.002865
2020-02-09    0.002660
2020-02-10    0.003110
2020-02-11    0.002365
2020-02-12    0.001780
2020-02-13    0.001620
2020-02-14    0.001560
2020-02-15    0.001460
Name: index, dtype: float64